*Первый шаг -- скрейпим нужные данные с сайта https://www.fragrantica.ru/.
Отмечу, что все импорты и установки сделаны под среду разработки Google Colab, где выполнялся этот этап проекта. В Google Colab код выполняется корректно но всех этапах. В других средах -- не тестировала.*

In [ ]:
# Сделаем необходимые установки и импорты.

!pip install pandas

!pip install requests

!pip install beautifulsoup4

!pip install fake_useragent

import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import time
from fake_useragent import UserAgent

# Для работы с файлами.
import csv
import glob



In [ ]:
# Загрузим страницу парфюмерного бренда, где содержатся нужные нам ссылки на отдельные для каждого парфюма страницы с отзывами.
# Если попробовать это сделать без использования UserAgent, то результатом будет ошибка 403 (доступ запрещен).
# С UserAgent - всё хорошо, статус - 200 (успешная загрузка).

url_brand = 'https://www.fragrantica.ru/designers/By-Kilian.html'
page = rq.get(url_brand, headers={'User-Agent': UserAgent().chrome})
print(page)

<Response [200]>


In [ ]:
# Используем наш основной инструмент для парсинга - Beautiful Soup.
soup_brand = BeautifulSoup(page.text, features="html.parser")

In [ ]:
# Cоздадим пустой список, куда мы будем помещать нужные нам ссылки.
# Эти ссылки лежат в тэге "a", но в этом тэге есть еще множество других, не нужных нам ссылок.
# Указываем дополнительный параметр поиска: ищем ссылки под родительским тэгом "heading 3".

urls_list = []
for link in soup_brand.find_all('a'):
    if link.parent.name == 'h3':
      urls_list.append(link.get('href'))
urls_list

['/perfume/By-Kilian/A-Kiss-from-a-Rose-66726.html',
 '/perfume/By-Kilian/A-Taste-of-Heaven-4317.html',
 '/perfume/By-Kilian/Apple-Brandy-22614.html',
 '/perfume/By-Kilian/Back-to-Black-6457.html',
 '/perfume/By-Kilian/Beyond-Love-4319.html',
 '/perfume/By-Kilian/Black-Phantom-43632.html',
 '/perfume/By-Kilian/Born-to-be-Unforgettable-89141.html',
 '/perfume/By-Kilian/Can-t-Stop-Loving-You-78735.html',
 '/perfume/By-Kilian/Cruel-Intentions-4320.html',
 '/perfume/By-Kilian/Dark-Lord-50603.html',
 '/perfume/By-Kilian/Do-It-For-Love-51438.html',
 '/perfume/By-Kilian/Good-Girl-Gone-Bad-Eau-Fraiche-59813.html',
 '/perfume/By-Kilian/Good-Girl-Gone-Bad-Extreme-43810.html',
 '/perfume/By-Kilian/Kologne-Shield-of-protection-73317.html',
 '/perfume/By-Kilian/Lemon-in-Zest-31399.html',
 '/perfume/By-Kilian/Liaisons-Dangereuses-4321.html',
 '/perfume/By-Kilian/Love-and-Tears-9951.html',
 '/perfume/By-Kilian/Love-Don-t-Be-Shy-4322.html',
 '/perfume/By-Kilian/Love-Don-t-Be-Shy-Extreme-64761.html',
 

In [ ]:
# Видим, что полученные нами в предыдущем шаге ссылки - корректны, однако они усеченные.
# Дополним их, чтоб они стали функциональны.

for i in range(len(urls_list)):
  urls_list[i] = 'https://www.fragrantica.ru/' + urls_list[i]

len(urls_list)

97

In [ ]:
# Создадим функцию, которая будет возращать нужные нам данные.
def GetInfo(url):
  page = rq.get(url, headers={'User-Agent': UserAgent().chrome})
  if page.status_code == 200: # Добавим страховку, на случай ошибок.
    soup = BeautifulSoup(page.text, features="html.parser")
    reviews = soup.find_all('div', {'itemprop' : 'review'}) # Загружаем контейнеры, которые содержат информацию об отзывах.
    reviews_full = []
    name_frag = soup.find('h1', {'itemprop' :'name'}).text # Найдем название парфюма.
    for review in reviews:
      date = review.find('span', {'itemprop' : 'datePublished'}).text # Найдем дату и время создания отзыва.
      author = review.find('b', {'class' : 'idLinkify'}).attrs['title'].replace('https://www.fragrantica.ru/chlen/', '') # Найдем ID пользователя, оставившего отзыв.
      body = review.find('div', {'itemprop' : 'reviewBody'}).text # Найдем непосредственно текст отзыва.
      review_item = [name_frag, date, author, body]
      reviews_full.append(review_item)
    return reviews_full
  else:
    return []

In [ ]:
# Проверим корректность работы функции.
GetInfo(urls_list[2])

[['Animalique Byredo для мужчин и женщин',
  '01/27/24 08:55',
  '224870',
  'Скажу только впечатление после затестов на коже: мягкий, замшево-пудровый с древесной ноткой и фиалкой. В прошлый раз слышала добавочно легкую ноту сандала как в Призраке пустыне, сегодня цветочную как ландыш Инфлоресенс. И аброксан есть , либо что-то на него похожее...животинка, но не сильно.'],
 ['Animalique Byredo для мужчин и женщин',
  '01/26/24 15:17',
  '377004',
  "Купаж из Accord oud + Immortel oud + Bal d'Afrique. У меня есть все три , поэтому для меня ничего нового. Как говориться кризис жанра...."],
 ['Animalique Byredo для мужчин и женщин',
  '01/26/24 12:11',
  '322980',
  'Очень современный и стильный аромат, однозначно попадает в современные тенденции, скорее всего станет хитом. Похож на многое и ни на что одновременно. \nНа моей коже открывается идеальной кожей чистой выделки, довольно быстро смягчается и уже проявляются тёплые цветочные ноты с мягкой телесностью и нежнейшей, еле заметной, сл

In [ ]:
# Начнем последовательно парсить одну страницу за другой и складывать данные в переменную.
# В процессе работы было обнаружено, что сайт позволяет парсить только 10 ссылок за один заход (даже с длительным ожиданием).
# В случае, когда я упорствовала с парсингом, status_code для всех страниц возвращался с ошибкой 429 (слишком много запросов).
# Добавляем достаточно длительное ожидание между итерациями, чтобы не тревожить сервер; для скачивания берем только сеты по 10 ссылок.

all_infos = []
for url in urls_list[90:97]:
  info = GetInfo(url)
  if len(info) != 0: all_infos.extend(info)
  time.sleep(120)

len(all_infos)

525

In [ ]:
# Отметим, что страница с отзывами оказалась динамической. Отзывы подгружаются при прокрутке, соответственно, с помощью BS мы их полностью вытащить не можем.
# В дальнейшем эту проблему можно будет пробовать решить через использование библиотеки Selenium, но на данном этапе мы решили собирать
# только те отзывы, которые не требуют прокрутки для загрузки.
# Достаточно много отзывов можно спарсить без прокрутки.
# Например, здесь (https://www.fragrantica.ru/perfume/Vilhelm-Parfumerie/Dear-Polly-30928.html) на 08.02. 24 - 262 отзыва.
# Нам  с помощью BS удалось получить 210.
all_infos

[['125th & Bloom Vilhelm Parfumerie для мужчин и женщин',
  '12/12/23 13:12',
  '349601',
  'Давно поняла, что это вцелом по композиции не моя марка. Роза некрасивая. Бывают такие? На блоттере превращается в мокрый картон. И вообще это про пыльные коробки, нос упирается в бетонную стену. Не нравится.'],
 ['125th & Bloom Vilhelm Parfumerie для мужчин и женщин',
  '10/24/23 00:37',
  '33715',
  'Уместнее было бы как-то совместить страницу с Harlem Bloom и 125th & Bloom, потому что это один и тот же аромат, а у Harlem Bloom больше отзывов. Выпущенный в 2017 году, Harlem Bloom был переименован в 125th & Bloom в 2018 году в связи с какими-то юридическими проблемами.\n\n125th & Bloom – это красивое сочетание розы и чуть дымной кожи. Немного маскулинно, кожаный аккорд напоминает кожу в Rose & Cuir FM.\n\nРоза, как основная нота, выбрана не случайно. Западный Гарлем – "розовый" район (125-я улица входит в его состав), именно там Фонд наследия роз (The Heritage Rose Foundation), занимаясь сохра

*После того, как мы собрали нужные данные, пишем их в csv.*

In [ ]:
import csv

In [ ]:
fields = ['fragrance', 'date', 'user ID', 'review body']
rows = all_infos

with open('kilian_collab(90-97).csv', 'w', encoding = 'utf8') as f:

    # using csv.writer method from CSV package
    write = csv.writer(f)

    write.writerow(fields)
    write.writerows(rows)

*Итогом предыдущего этапа стали CSV файлы в количестве 22 шт. Далее мы их объединяем. Вручную это не делать не хочется, воспользуемся glob и pandas*

In [ ]:
import glob

csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['kilian_all.csv', 'byredo_all.csv', 'vilhelm_all.csv']

In [ ]:
import pandas as pd

df_csv_append = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(file)
    df_csv_append = df_csv_append.append(df, ignore_index=True)

df_csv_append

<ipython-input-13-6cb7609b1404>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_csv_append = df_csv_append.append(df, ignore_index=True)
<ipython-input-13-6cb7609b1404>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_csv_append = df_csv_append.append(df, ignore_index=True)
<ipython-input-13-6cb7609b1404>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_csv_append = df_csv_append.append(df, ignore_index=True)


,Unnamed: 0,fragrance,date,user ID,review body
0,0,Do It For Love By Kilian для мужчин и женщин,07/05/23 07:18,366958,"На мне уходит в мыло, в самое обычное цветочно..."
1,1,Do It For Love By Kilian для мужчин и женщин,03/20/23 08:46,39039,Добыла в коллекцию этот красивущий белоцветочн...
2,2,Do It For Love By Kilian для мужчин и женщин,08/03/22 12:14,223449,Шла вообще за другим :)\nОбычно к ароматам Кил...
3,3,Do It For Love By Kilian для мужчин и женщин,12/07/21 06:06,284612,"Скажу честно, я очень не люблю отзывы, где пиш..."
4,4,Do It For Love By Kilian для мужчин и женщин,11/26/20 06:01,276743,"Пвродоксальный момент, но на мне он не дал цит..."
...,...,...,...,...,...
14386,1995,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,02/04/20 04:16,195039,"При первом же вздохе поняла, что это брат Серж..."
14387,1996,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,08/22/20 12:50,177386,"Восхитительно красивый, слегка ностальгический..."
14388,1997,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,11/28/21 05:04,266684,Возможно по моим рецепторам проехался катом ко...
14389,1998,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,01/19/19 08:23,51446,Маринованный имбирь и васабииз японской кухни ...


In [ ]:
del df_csv_append['Unnamed: 0']


In [ ]:
df_csv_append

,fragrance,date,user ID,review body
0,Do It For Love By Kilian для мужчин и женщин,07/05/23 07:18,366958,"На мне уходит в мыло, в самое обычное цветочно..."
1,Do It For Love By Kilian для мужчин и женщин,03/20/23 08:46,39039,Добыла в коллекцию этот красивущий белоцветочн...
2,Do It For Love By Kilian для мужчин и женщин,08/03/22 12:14,223449,Шла вообще за другим :)\nОбычно к ароматам Кил...
3,Do It For Love By Kilian для мужчин и женщин,12/07/21 06:06,284612,"Скажу честно, я очень не люблю отзывы, где пиш..."
4,Do It For Love By Kilian для мужчин и женщин,11/26/20 06:01,276743,"Пвродоксальный момент, но на мне он не дал цит..."
...,...,...,...,...
14386,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,02/04/20 04:16,195039,"При первом же вздохе поняла, что это брат Серж..."
14387,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,08/22/20 12:50,177386,"Восхитительно красивый, слегка ностальгический..."
14388,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,11/28/21 05:04,266684,Возможно по моим рецепторам проехался катом ко...
14389,Stockholm 1978 Vilhelm Parfumerie для мужчин и...,01/19/19 08:23,51446,Маринованный имбирь и васабииз японской кухни ...


In [ ]:
df_csv_append.to_csv('FRAG_all.csv', encoding='utf-8')


В завершении первого этапа проекта мы получили файл, содержащий **14391** отзыв.